In [63]:
import pandas as pd
import numpy as np

In [64]:
vs_df = pd.read_csv('VS Product Archive 17_05_2024.csv', dtype=str)
vs_df.dropna(subset=['Reference'], inplace=True)
vs_df = vs_df[['Products', 'Reference']]
vs_df.rename(columns={'Reference': 'Root PLU'}, inplace=True)
vs_df.drop_duplicates(subset=['Root PLU'], keep='first', inplace=True)
vs_df.head()

,Products,Root PLU
0,The Bridge 2014 Cartella Brown,00019800
3,Ashwood Body Bag Black,00020190
6,Sienna De Luca Cow Fossil Black,00013028
9,Ashwood Hipster Trousers Black,00019642
12,Ashwood Hand-held Frolable Suit Carrier Brown,00020231


In [65]:
vs_df.shape

(3185, 2)

In [66]:
df_brands = pd.read_csv("Brands - Fashione.csv", skiprows=1, dtype=str)
df_brands = df_brands[df_brands['Season'] == 'Totals'].reset_index(drop=True)
df_brands['Style'] = df_brands['Brand']
df_brands = df_brands[['Style', 'Brand']]
all_brands = df_brands['Brand'].unique()

df = pd.read_csv("All brands - Fashione.csv", skiprows=4, dtype=str)
df.reset_index(inplace=True)
df2 = df[df['PLU'].isnull()]
df1 = df[~df['PLU'].isnull()]
df2 = df2[df2['Style'].isin(all_brands)]
df2 = df2.drop_duplicates(subset=['Style'], keep='last')
df2 = pd.merge(df2, df_brands, how='left', on='Style')
df = pd.concat([df1, df2], ignore_index=True)
df = df.sort_values(by='index')
df = df[df['PLU'] != 'PLU']
df.drop(columns=['index'], axis=1, inplace=True)
df['Brand'] = df['Brand'].bfill()
df = df[~df['PLU'].isnull()].reset_index(drop=True)
df['Units'] = df['Units'].astype(np.int64)
df['Root PLU'] = df['PLU'].str[:-2]
temp_df = df.groupby(['Root PLU'])['Units'].sum().reset_index()
temp_df.head()

,Root PLU,Units
0,00000001,0
1,00000015,0
2,00000029,0
3,00000043,0
4,00000057,0


In [67]:
vs_df = pd.merge(vs_df, temp_df, how='left', on='Root PLU')
vs_df.head()

,Products,Root PLU,Units
0,The Bridge 2014 Cartella Brown,00019800,NaN
1,Ashwood Body Bag Black,00020190,NaN
2,Sienna De Luca Cow Fossil Black,00013028,NaN
3,Ashwood Hipster Trousers Black,00019642,0.0
4,Ashwood Hand-held Frolable Suit Carrier Brown,00020231,6.0


In [68]:
vs_df.shape

(3185, 3)

In [69]:
list_delete_root_plu = vs_df[vs_df['Units'] == 0.0]['Root PLU'].unique()

In [70]:
delete_df = df[df['Root PLU'].isin(list_delete_root_plu)]
delete_df.head()

,Style,Unnamed: 1,Colour,Unnamed: 3,Unnamed: 4,Unnamed: 5,Size,Unnamed: 7,Description,Unnamed: 9,...,Category,Unnamed: 12,PLU,Unit Cost,Unit Retail,Units,Total Cost,Total Retail,Brand,Root PLU
0,AL FRANCO,NaN,BLACK,NaN,NaN,NaN,M,NaN,criss cross safari,NaN,...,MENS JACKETS,NaN,0001727504,70.00,349.00,0,0.00,0.00,AL FRANCO,00017275
1,AL FRANCO,NaN,BLACK,NaN,NaN,NaN,L,NaN,criss cross safari,NaN,...,MENS JACKETS,NaN,0001727505,70.00,349.00,0,0.00,0.00,AL FRANCO,00017275
2,AL FRANCO,NaN,BLACK,NaN,NaN,NaN,XL,NaN,criss cross safari,NaN,...,MENS JACKETS,NaN,0001727506,70.00,349.00,0,0.00,0.00,AL FRANCO,00017275
3,AL FRANCO,NaN,BLACK,NaN,NaN,NaN,2XL,NaN,criss cross safari,NaN,...,MENS JACKETS,NaN,0001727507,70.00,349.00,0,0.00,0.00,AL FRANCO,00017275
4,AL FRANCO,NaN,BLACK,NaN,NaN,NaN,3XL,NaN,criss cross safari,NaN,...,MENS JACKETS,NaN,0001727508,70.00,369.00,0,0.00,0.00,AL FRANCO,00017275


In [71]:
df.shape, delete_df.shape

((18060, 21), (4879, 21))

In [72]:
delete_df[['Style', 'Colour', 'Size', 'Description', 'Category', 'PLU', 'Unit Cost', 'Unit Retail', 'Units', 'Brand']].to_csv('Delete SKUs from Fashione - VS Product Archive.csv', index=False)

In [74]:
delete_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4879 entries, 0 to 18051
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Style         4879 non-null   object
 1   Unnamed: 1    0 non-null      object
 2   Colour        4875 non-null   object
 3   Unnamed: 3    0 non-null      object
 4   Unnamed: 4    0 non-null      object
 5   Unnamed: 5    0 non-null      object
 6   Size          4097 non-null   object
 7   Unnamed: 7    0 non-null      object
 8   Description   4879 non-null   object
 9   Unnamed: 9    0 non-null      object
 10  Unnamed: 10   0 non-null      object
 11  Category      4879 non-null   object
 12  Unnamed: 12   0 non-null      object
 13  PLU           4879 non-null   object
 14  Unit Cost     4879 non-null   object
 15  Unit Retail   4879 non-null   object
 16  Units         4879 non-null   int64 
 17  Total Cost    4879 non-null   object
 18  Total Retail  4879 non-null   object
 19  Brand     